# Age and gender detection

Using a deep learning model as a background we can recognize the gender and the age of a face.

One of the model used to detect gender and age was created and trained by Gil Levi and Tal Hassner. They used the Adidence dataset, which has 26,850 photos from 2,284 subjects. They classified, for gender between male and female, and for age in range of 0-2, 4-6, 8-13, 15-20, 25-32, 38-43, 48-53, 60-.

The files used are:

* Age classification Caffe model.
* Age classification protext file.
* Gender classification Caffe model.
* Gender classification protext file.
* Age classification Caffe model.

Caffe is a Deep Learning framework, and protext is actually a configuration file which tell a Caffe model how the neural network want to be trained. After training the model, we will get the trained model in a file with extension .caffemodel.

## Implementation

#### Import libraries

In [1]:
import cv2
import face_recognition

### Video from real time webcam

In [6]:
# Get the default webcam video
webcam_video_stream = cv2.VideoCapture(0)

# Initialize empty array for face locations
all_face_locations = []

# Loop through every frame in the video
while True:
    # Get the current frame from the video stream as an image
    ret, current_frame = webcam_video_stream.read()
    # Resize the current frame to 1/4 size to proces faster
    current_frame_small = cv2.resize(current_frame, (0, 0), fx = 0.25, fy = 0.25)
    # Detect all faces in the image
    all_face_locations = face_recognition.face_locations(current_frame_small, model = "hog")

    # Looping through the face locations
    for index, current_face_location in enumerate(all_face_locations):
        top_position, right_position, bottom_position, left_position = current_face_location
        # Change the position magnitude to fit the actual size video frame
        top_position = top_position * 4
        right_position = right_position * 4
        bottom_position = bottom_position * 4
        left_position = left_position * 4

        # Slicing the current face from main image
        current_face_image = current_frame[top_position:bottom_position, left_position:right_position]

        # The "Age_Gender_Model_Mean_Values" calculated by using the numpy.mean()
        Age_Gender_Model_Mean_Values = (78.4263377603, 87.7689143744, 114.895847746)

        # Create the binary large object (blob) of our face slice
        current_face_image_blob = cv2.dnn.blobFromImage(current_face_image, 1, (227, 227), Age_Gender_Model_Mean_Values)

        ##################### Gender #####################

        # Declare gender labels, protext and caffemodel file paths
        gender_label_list = ["Male", "Female"]
        gender_protext = ('../Models/Gender and age model/gender_deploy.prototxt')
        gender_caffemodel = ('../Models/Gender and age model/gender_net.caffemodel')

        # Create model from files and provide blob as input
        gender_cov_net = cv2.dnn.readNet(gender_caffemodel, gender_protext)
        gender_cov_net.setInput(current_face_image_blob) 

        # Get gender predictions and get label of maximum value returned item
        gender_predictions = gender_cov_net.forward()
        gender = gender_label_list[gender_predictions[0].argmax()]

        ##################### Age #####################

        # Declare age labels, protext and caffemodel file paths
        age_label_list = ["(0-2)", "(4-6)", "(8-12)", "(15-20)", "(25-32)", "(38-43)", "(48-53)", "(60-100)"]
        age_protext = ('../Models/Gender and age model/age_deploy.prototxt')
        age_caffemodel = ('../Models/Gender and age model/age_net.caffemodel')

        # Create model from files and provide blob as input
        age_cov_net = cv2.dnn.readNet(age_caffemodel, age_protext)
        age_cov_net.setInput(current_face_image_blob) 

        # Get gender predictions and get label of maximum value returned item
        age_predictions = age_cov_net.forward()
        age = age_label_list[age_predictions[0].argmax()]

        # Draw rectangle around the face detected
        cv2.rectangle(current_frame, (left_position, top_position), (right_position, bottom_position), (0, 0, 255), 2)

        # Display the name as text in the image
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(current_frame, gender + " " + age + "years", (left_position, bottom_position + 20), font, 0.5, (0, 255, 0), 1)

    # Showing the current fae with rectangle drawn
    cv2.imshow("Webcam Video", current_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

webcam_video_stream.release()
cv2.destroyAllWindows() 

### Image age and gender detection

In [2]:
# Load the image to detect
image_detect = cv2.imread("../Images/Testing/Image 10.jpg")

# Detect all faces in the image
all_face_locations = face_recognition.face_locations(image_detect, model = "cnn")

# Print the number of faces detected
print("There are {} faces in this image".format(len(all_face_locations)))

for index, current_face_location in enumerate(all_face_locations):
    top_position, right_position, bottom_position, left_position = current_face_location
    print("Found face {} at top: {}, right: {}, bottom: {}, left: {}".format(index + 1, top_position, right_position, bottom_position, left_position))
    current_face_image = image_detect[top_position:bottom_position, left_position:right_position]

    # The "Age_Gender_Model_Mean_Values" calculated by using the numpy.mean()
    Age_Gender_Model_Mean_Values = (78.4263377603, 87.7689143744, 114.895847746)

    # Create the binary large object (blob) of our face slice
    current_face_image_blob = cv2.dnn.blobFromImage(current_face_image, 1, (227, 227), Age_Gender_Model_Mean_Values, swapRB=False)

    ##################### Gender #####################

    # Declare gender labels, protext and caffemodel file paths
    gender_label_list = ["Male", "Female"]
    gender_protext = ('../Models/Gender and age model/gender_deploy.prototxt')
    gender_caffemodel = ('../Models/Gender and age model/gender_net.caffemodel')

    # Create model from files and provide blob as input
    gender_cov_net = cv2.dnn.readNet(gender_caffemodel, gender_protext)
    gender_cov_net.setInput(current_face_image_blob) 

    # Get gender predictions and get label of maximum value returned item
    gender_predictions = gender_cov_net.forward()
    gender = gender_label_list[gender_predictions[0].argmax()]

    ##################### Age #####################

    # Declare age labels, protext and caffemodel file paths
    age_label_list = ["(0-2)", "(4-6)", "(8-12)", "(15-20)", "(25-32)", "(38-43)", "(48-53)", "(60-100)"]
    age_protext = ('../Models/Gender and age model/age_deploy.prototxt')
    age_caffemodel = ('../Models/Gender and age model/age_net.caffemodel')

    # Create model from files and provide blob as input
    age_cov_net = cv2.dnn.readNet(age_caffemodel, age_protext)
    age_cov_net.setInput(current_face_image_blob) 

    # Get gender predictions and get label of maximum value returned item
    age_predictions = age_cov_net.forward()
    age = age_label_list[age_predictions[0].argmax()]

    # Draw rectangle around the face detected
    cv2.rectangle(image_detect, (left_position, top_position), (right_position, bottom_position), (0, 0, 255), 2)

    # Display the name as text in the image
    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.putText(image_detect, gender + " " + age + "years", (left_position, bottom_position + 20), font, 0.5, (0, 255, 0), 1)

# Showing the current fae with rectangle drawn
cv2.imshow("Age and Gender", image_detect)
cv2.waitKey(0)
cv2.destroyAllWindows()

There are 1 faces in this image
Found face 1 at top: 34, right: 174, bottom: 91, left: 117
